In [ ]:
!git clone --branch dev https://github.com/rolysr/medical-knowledge-discoverer

Cloning into 'medical-knowledge-discoverer'...
remote: Enumerating objects: 243, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 243 (delta 49), reused 86 (delta 30), pack-reused 136
Receiving objects: 100% (243/243), 1.17 MiB | 19.94 MiB/s, done.
Resolving deltas: 100% (89/89), done.


In [ ]:
%cd medical-knowledge-discoverer/
!ls

/content/medical-knowledge-discoverer
datasets  main.py  ontology   requirements.txt
LICENSE   models   README.md  utils


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.0/177.0 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 108.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of es-core-news-sm to determine which version is compatible with other requirements. This cou

In [ ]:
from pathlib import Path
from copy import copy
from utils.anntools import Collection
from re import sub
import csv

def get_marked_sentence_t5_input_format(sentence, ent1, label1, ent2, label2):
  new_sentence = copy(sentence)
  new_sentence = new_sentence.replace(ent1, "<{0}1>{1}</{0}1>".format(label1, ent1))
  new_sentence = new_sentence.replace(ent2, "<{0}2>{1}</{0}2>".format(label2, ent2))
  return new_sentence

def get_t5_output_format(ent1, label1, ent2, label2, relation_label):
  return relation_label + "({}, {})".format(label1+"1", label2+"2")

def generate_t5_input_output_format(collection):
  dataset = []
  for sentence in collection.sentences:
    for relation in sentence.relations:
        origin = relation.from_phrase
        origin_text = origin.text.lower()
        destination = relation.to_phrase
        destination_text = destination.text.lower()

        #making the pair
        input_text = get_marked_sentence_t5_input_format(sentence.text, origin_text, origin.label, destination_text, destination.label)
        output_text = get_t5_output_format(origin_text, origin.label, destination_text, destination.label, relation.label)
        dataset.append((input_text, output_text))
  return dataset

ref = Path('./datasets/train')
collection = Collection().load_dir(ref)
train_dataset = generate_t5_input_output_format(collection)
MAX_INPUT_TOKENS = max([len(data[0]) for data in train_dataset])
MAX_OUTPUT_TOKENS = max([len(data[1]) for data in train_dataset])

In [ ]:
print(MAX_INPUT_TOKENS, MAX_OUTPUT_TOKENS)

524 36


In [ ]:
for data in train_dataset:
  print(data)

Streaming output truncated to the last 5000 lines.
('Los pólipos pueden ser extirpados cuando un médico examina el <Predicate1>interior</Predicate1> del <Concept2>intestino grueso</Concept2> durante una colonoscopía.', 'domain(Predicate1, Concept2)')
('Los pólipos pueden ser extirpados cuando un médico examina el interior del intestino grueso <Action1>durante</Action1> una <Concept2>colonoscopía</Concept2>.', 'target(Action1, Concept2)')
('Los pólipos pueden ser <Action2>extirpados</Action2> cuando un médico <Action1>examina</Action1> el interior del intestino grueso durante una colonoscopía.', 'entails(Action1, Action2)')
('El cáncer de pulmón es uno de los cánceres <Predicate1>más</Predicate1> comunes en el <Concept2>mundo</Concept2>.', 'in-place(Predicate1, Concept2)')
('El cáncer de pulmón es uno de los <Concept2>cánceres</Concept2> <Predicate1>más</Predicate1> comunes en el mundo.', 'domain(Predicate1, Concept2)')
('El cáncer de pulmón es uno de los cánceres <Predicate1>más</Predi

In [ ]:
# store relations for training in re_train.csv
with open('re_train.csv', 'w', newline='') as file:
  writer = csv.writer(file)
  writer.writerow(["input", "output"])
  for data in train_dataset:
    input = str(data[0])
    output = str(data[1])
    writer.writerow([input, output])

In [ ]:
!pip install simplet5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# let's get a dataset
import pandas as pd
from sklearn.model_selection import train_test_split

path = "re_train.csv"
df = pd.read_csv(path)
df.head()

,input,output
0,The current outbreak of <Concept1>coronavirus ...,"same-as(Concept1, Concept2)"
1,The current <Concept1>outbreak</Concept1> of <...,"in-context(Concept1, Concept2)"
2,The current outbreak of coronavirus disease-20...,"has-property(Concept1, Concept2)"
3,The current outbreak of coronavirus disease-20...,"same-as(Concept1, Concept2)"
4,The current <Concept2>outbreak</Concept2> of c...,"causes(Concept1, Concept2)"


In [ ]:
# simpleT5 expects dataframe to have 2 columns: "source_text" and "target_text"
df = df.rename(columns={"output":"target_text", "input":"source_text"})
df = df[['source_text', 'target_text']]

# T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
df['source_text'] = df['source_text']
df

,source_text,target_text
0,The current outbreak of <Concept1>coronavirus ...,"same-as(Concept1, Concept2)"
1,The current <Concept1>outbreak</Concept1> of <...,"in-context(Concept1, Concept2)"
2,The current outbreak of coronavirus disease-20...,"has-property(Concept1, Concept2)"
3,The current outbreak of coronavirus disease-20...,"same-as(Concept1, Concept2)"
4,The current <Concept2>outbreak</Concept2> of c...,"causes(Concept1, Concept2)"
...,...,...
8403,La obesidad aumenta el riesgo de <Action1>pade...,"target(Action1, Concept2)"
8404,La obesidad aumenta el riesgo de <Action1>pade...,"target(Action1, Concept2)"
8405,La obesidad aumenta el riesgo de <Action1>pade...,"target(Action1, Concept2)"
8406,La obesidad aumenta el riesgo de <Action1>pade...,"target(Action1, Predicate2)"


In [ ]:
train_df, test_df = train_test_split(df, test_size=0.1)
train_df.shape, test_df.shape

((7567, 2), (841, 2))

In [ ]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
model.train(train_df=train_df,
            eval_df=test_df, 
            source_max_token_len=MAX_INPUT_TOKENS + 50, 
            target_max_token_len=MAX_OUTPUT_TOKENS + 8, 
            batch_size=8, max_epochs=4, use_gpu=True)

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

...

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
!ls
!ls ./outputs

datasets  lightning_logs  models    outputs    requirements.txt  utils
LICENSE   main.py	  ontology  README.md  re_train.csv
simplet5-epoch-0-train-loss-0.3134-val-loss-0.1507
simplet5-epoch-1-train-loss-0.1503-val-loss-0.114
simplet5-epoch-2-train-loss-0.1107-val-loss-0.0944
simplet5-epoch-3-train-loss-0.0877-val-loss-0.0903


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
# let's load the trained model for testing:
model.load_model("t5","./outputs/simplet5-epoch-3-train-loss-0.0877-val-loss-0.0903", use_gpu=True)

In [70]:
ref = Path('./datasets/test/scenario1-main')
collection = Collection().load_dir(ref)

test_dataset = generate_t5_input_output_format(collection)
for data in test_dataset:
  print(data)

('Los glóbulos blancos ayudan a su <Concept2>organismo</Concept2> a <Action1>combatir</Action1> infecciones.', 'subject(Action1, Concept2)')
('Los glóbulos blancos ayudan a su organismo a <Action1>combatir</Action1> <Concept2>infecciones</Concept2>.', 'target(Action1, Concept2)')
('Los <Concept2>glóbulos blancos</Concept2> <Action1>ayudan</Action1> a su organismo a combatir infecciones.', 'subject(Action1, Concept2)')
('Los glóbulos blancos <Action1>ayudan</Action1> a su organismo a <Action2>combatir</Action2> infecciones.', 'target(Action1, Action2)')
('Se contagia a través de relaciones sexuales con una <Concept1>persona</Concept1> <Concept2>infectada</Concept2>.', 'in-context(Concept1, Concept2)')
('Se contagia a través de <Concept1>relaciones sexuales</Concept1> con una <Concept2>persona</Concept2> infectada.', 'in-context(Concept1, Concept2)')
('Se <Action1>contagia</Action1> a través de relaciones <Reference2>se</Reference2>xuales con una persona infectada.', 'target(Action1, Ref

In [47]:
# store relations for training in re_train.csv
with open('re_test.csv', 'w', newline='') as file:
  writer = csv.writer(file)
  writer.writerow(["input", "output"])
  for data in test_dataset:
    input = str(data[0])
    output = str(data[1])
    writer.writerow([input, output])

In [54]:
path = "re_test.csv"
test_df = pd.read_csv(path)
test_df.head()

,input,output
0,Los glóbulos blancos ayudan a su <Concept2>org...,"subject(Action1, Concept2)"
1,Los glóbulos blancos ayudan a su organismo a <...,"target(Action1, Concept2)"
2,Los <Concept2>glóbulos blancos</Concept2> <Act...,"subject(Action1, Concept2)"
3,Los glóbulos blancos <Action1>ayudan</Action1>...,"target(Action1, Action2)"
4,Se contagia a través de relaciones sexuales co...,"in-context(Concept1, Concept2)"


In [55]:
# simpleT5 expects dataframe to have 2 columns: "source_text" and "target_text"
test_df = test_df.rename(columns={"output":"target_text", "input":"source_text"})
test_df = test_df[['source_text', 'target_text']]

# T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
test_df['source_text'] = test_df['source_text']
test_df

,source_text,target_text
0,Los glóbulos blancos ayudan a su <Concept2>org...,"subject(Action1, Concept2)"
1,Los glóbulos blancos ayudan a su organismo a <...,"target(Action1, Concept2)"
2,Los <Concept2>glóbulos blancos</Concept2> <Act...,"subject(Action1, Concept2)"
3,Los glóbulos blancos <Action1>ayudan</Action1>...,"target(Action1, Action2)"
4,Se contagia a través de relaciones sexuales co...,"in-context(Concept1, Concept2)"
...,...,...
839,The <Concept1>sequence identity</Concept1> bet...,"has-property(Concept1, Concept2)"
840,The sequence identity between the 3CL protease...,"in-context(Concept1, Concept2)"
841,The sequence identity between the 3CL protease...,"in-context(Concept1, Concept2)"
842,The sequence identity <Predicate1>between</Pre...,"arg(Predicate1, Concept2)"


In [75]:
model.predict("Los glóbulos blancos ayudan a su <Concept2>organismo</Concept2> a <Action1>combatir</Action1> infecciones.")

['subject(Action1, Concept2)']

In [77]:
def eval_model(collection):
  CORRECT, MISSING, SPURIOUS = 0, 0, 0
  counter = 0
  for sentence in collection.sentences:
    counter += 1
    print('Evaluating sentence', counter)
    for relation in sentence.relations:
        origin = relation.from_phrase
        origin_text = origin.text.lower()
        destination = relation.to_phrase
        destination_text = destination.text.lower()

        #making the pair
        input_text = get_marked_sentence_t5_input_format(sentence.text, origin_text, origin.label, destination_text, destination.label)
        output_text = get_t5_output_format(origin_text, origin.label, destination_text, destination.label, relation.label)
        if model.predict(input_text)[0] == output_text:
          CORRECT += 1
        else:
          SPURIOUS += 1
    entities_number = len(sentence.keyphrases)
    max_relations_number = entities_number*(entities_number + 1) / 2
    MISSING += max_relations_number - len(sentence.relations)
  rec_b = CORRECT/(CORRECT + MISSING)
  prec_b =  CORRECT/(CORRECT + SPURIOUS)
  f1_b = (prec_b*rec_b)/(prec_b+rec_b)
  return rec_b, prec_b, f1_b

ref = Path('./datasets/test/scenario1-main')
collection = Collection().load_dir(ref)
rec_b, prec_b, f1_b = eval_model(collection)
print("REC_B: ", rec_b)
print("PREC_B: ", prec_b)
print("F1_B: ", f1_b)

Evaluating sentence 1
Evaluating sentence 2
Evaluating sentence 3
Evaluating sentence 4
Evaluating sentence 5
Evaluating sentence 6
Evaluating sentence 7
Evaluating sentence 8
Evaluating sentence 9
Evaluating sentence 10
Evaluating sentence 11
Evaluating sentence 12
Evaluating sentence 13
Evaluating sentence 14
Evaluating sentence 15
Evaluating sentence 16
Evaluating sentence 17
Evaluating sentence 18
Evaluating sentence 19
Evaluating sentence 20
Evaluating sentence 21
Evaluating sentence 22
Evaluating sentence 23
Evaluating sentence 24
Evaluating sentence 25
Evaluating sentence 26
Evaluating sentence 27
Evaluating sentence 28
Evaluating sentence 29
Evaluating sentence 30
Evaluating sentence 31
Evaluating sentence 32
Evaluating sentence 33
Evaluating sentence 34
Evaluating sentence 35
Evaluating sentence 36
Evaluating sentence 37
Evaluating sentence 38
Evaluating sentence 39
Evaluating sentence 40
Evaluating sentence 41
Evaluating sentence 42
Evaluating sentence 43
Evaluating sentence 